TODO
1. develop kernel 
2. validate kernel with h-param variation as in ref#2

In [1]:
import numpy as np
import pandas as pd
import logging
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr
from numpy.linalg import cholesky, det, lstsq,inv
logging.basicConfig(level=logging.INFO)

In [2]:
def kernel(xp,xq,p,q,lambda_diag,v,sigma_n):
    logger = logging.getLogger()
    logger.setLevel(level=logging.DEBUG)
    """
    xp: float array mx1 vector of input point p
    xq : float array
    lambda_ 1Xm represents the diagonal of the hyperparemter diagonal matrix lambda_
    v: hyperparemter
    sigma2_n : hyperparameter
    """
    #TODO check all inputes are doubles
    lambda_power_minus_1 = np.diag(np.reciprocal(lambda_diag))
    exponent_ = -0.5*np.matmul(np.matmul((xp-xq).T,lambda_power_minus_1),(xp-xq))
    delta_pq = 1.0 if p==q else 0.0
    kernel = v**2 * np.exp(exponent)+delta_pq*sigma_n**2
    #logging
    logger.debug(f'lambda diag= {lambda_diag}')
    logger.debug(f'lambda_pow_minus_1 {lambda_power_minus_1}')
    logger.debug(f'term1 = -(xp-xq).T * lambda_pow_minus_1 = {term1}')
    logger.debug(f'exponent = {exponent_}')
    ######
    return kernel

In [3]:
# don't delete
xp = pd.Series([1.0,2.0]).values.reshape(-1,1)
xq = pd.Series([1.0,2.0]).values.reshape(-1,1)
lambda_diag = [0.2,0.3]
lambda_power_minus_1 = np.diag(np.reciprocal(lambda_diag))
lambda_power_minus_1
(xp-xq).T.shape
term1 = np.matmul(-(xp-xq).T,lambda_power_minus_1)
exponent = np.matmul(term1,(xp-xq))

In [4]:
k= kernel(xp=xp,xq=xq,p=1,q=2,lambda_diag=lambda_diag,sigma_n=0.05,v=1.0)
k

DEBUG:root:lambda diag= [0.2, 0.3]
DEBUG:root:lambda_pow_minus_1 [[5.         0.        ]
 [0.         3.33333333]]
DEBUG:root:term1 = -(xp-xq).T * lambda_pow_minus_1 = [[0. 0.]]
DEBUG:root:exponent = [[-0.]]


array([[1.]])

In [5]:
# quick validation :kernel vs euclidean distance , comment out , time consuming
# dim = 100,5
# X = np.random.normal(loc=0,scale=1.0,size=dim)
# k = np.zeros(dim[0]**2)
# euc = np.zeros(dim[0]**2)
# idx = 0
# for i in np.arange(dim[0]):
#     for j in np.arange(dim[0]):
#         xp = X[i,].reshape(-1,1)
#         xq = X[j,].reshape(-1,1)
#         k[idx] = kernel(xp=xp,xq=xq,p=i,q=j,lambda_diag=[0.2,0.1,0.3,0.4,0.5],sigma_n=0.05,v=1.0)
#         euc[idx] = euclidean(u=xp,v=xq)
#         idx = idx + 1
# pearsonr(x=k,y=np.reciprocal(euc+0.01))
# if corr is +ve and significant, then we are good

In [6]:
# test scipy minimize

In [7]:
from scipy.optimize import minimize
def fn(x,x0,c):
    return np.sum((x-x0)**2)+c
x0 = np.array([[1,1]])
x = np.array([[2,2]])

x_min = minimize(fun=fn,x0=np.array([[0,0]]),args=(x0,10))
x_min.x

array([1., 1.])

In [8]:
# def kernel as matrix operation

In [9]:
# Note: in http://krasserm.github.io/2018/03/19/gaussian-processes/
# in the implementatoin of Gaussian kernel , the sum of the two squared terms is broadcast sum
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.add.html
# https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html
# why ? as in first term , reshape(-1,1) is used , then the row sum shape is converted from (m,) to (m,1)
# then it is summed with array of shape (n,) then shapes are not equal, even if m==n , then the second dimensions 
# are not equal
import numpy as np
def kernel(X1, X2, l=1.0, sigma_f=1.0):
    ''' Isotropic squared exponential kernel. Computes a covariance matrix from points in X1 and X2. Args: X1: Array of m points (m x d). X2: Array of n points (n x d). Returns: Covariance matrix (m x n). '''
    # the sume is broad cast 
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

X1 = np.random.normal(loc = 10,scale = 2, size = (10,2))
X2 = np.random.normal(loc = 10,scale = 2, size = (10,2))
X1_sum= np.sum(X1**2, 1).reshape(-1,1)
X2_sum= np.sum(X2**2, 1)
X2_sum_2= np.sum(X2**2, 1).reshape(-1,1)
tot = X1_sum + X2_sum
print((X1_sum + X2_sum).shape)
print((X1_sum + X2_sum_2).shape)

(10, 10)
(10, 1)


In [10]:
def kernel_mtx(X1,X2,lambda_diag,v,sigma_n):
    ## from http://krasserm.github.io/2018/03/19/gaussian-processes/
    ## sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    
    logger = logging.getLogger()
    logger.setLevel(level=logging.INFO)
    assert X1.shape[1] == X2.shape[1]
    assert X1.shape[1] == len(lambda_diag)
    
    
    lambda_power_minus_1 = np.diag(np.reciprocal(lambda_diag))
    # quadratic terms are scale by inverse precision (variance) , precision = lambda (Captial case)
    X1_sq_sum_scaled = np.sum(a=np.matmul(X1**2,lambda_power_minus_1),axis=1)
    X2_sq_sum_scale = np.sum(a=np.matmul(X2**2,lambda_power_minus_1),axis=1)
    X1_matmul_X2_T_scaled = np.matmul(np.matmul(X1,lambda_power_minus_1),X2.T)
    
    sqdist_scaled = X1_sq_sum_scaled.reshape(-1,1)+X2_sq_sum_scale-2*X1_matmul_X2_T_scaled
    sigma_n_sq = np.zeros((X1.shape[0],X2.shape[0]))
    if X1.shape[0] == X2.shape[0]: #same data-set , or at least comparable
        sigma_n_sq = np.diag(np.repeat(sigma_n**2,X1.shape[0]))
        
    kernel = v**2 * np.exp(-0.5*sqdist_scaled)+sigma_n_sq
    logger.debug(f'sqdist_scaled = {sqdist_scaled}')
    return kernel


M = 10
N = 8
D = 3
X1 = np.random.normal(loc = 10, scale  = 2, size = (M,D))
X2 = np.random.normal(loc = 5, scale  = 1, size = (N,D))
lambda_diag = np.random.normal(loc = 1, scale = 0.01, size = D)
v = 1
sigma_n = 0.05
X1.shape
k = kernel_mtx(X1,X1,lambda_diag,v,sigma_n)
np.diag(k)

array([1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025, 1.0025,
       1.0025, 1.0025])

In [11]:
# packing theta
def kernel_fn(theta,X1,X2):
    assert X1.shape[1] == X2.shape[1]
    D = X1.shape[1]
    lambda_diag=theta[0:D]
    v= theta[D]
    sigma_n = theta[D+1]
    return kernel_mtx(X1,X1,lambda_diag,v,sigma_n)

In [12]:
Y1 = np.random.normal(loc=2,scale = 1,size=(M,2))
theta_0 = np.random.normal(loc=2,scale = 1,size=X1.shape[1]+2)
def neg_ll(theta,X,t,kernel_fn):
    assert X.shape[0]==t.shape[0]
    K = kernel_fn(theta,X,X)
    #Y_train.T.dot(inv(K).dot(Y_train)
    neg_ll_tot = 0
    for j in np.arange(t.shape[1]):    
        neg_ll_j = np.log(det(K))+t[:,j].T.dot(inv(K).dot(t[:,j]))
        neg_ll_tot = neg_ll_tot+neg_ll_j
    return neg_ll_tot
t = np.random.normal(loc=5,scale = 1,size = (M,1))
neg_ll(X=X1,t=t,kernel_fn=kernel_fn,theta=theta_0)

43.3733470976922

In [13]:
minimize(fun=neg_ll,x0=theta_0,args=(X1,t,kernel_fn))

      fun: 15.21885460868222
 hess_inv: array([[ 5.52464269,  1.55265189,  4.08385568, -3.40913393,  0.0895512 ],
       [ 1.55265197,  2.78212314,  2.45071527,  1.12476522,  0.01832404],
       [ 4.08385563,  2.45071538,  5.56792602, -1.12412138,  0.08199106],
       [-3.40913393,  1.12476522, -1.12412138,  6.83022595, -0.07083123],
       [ 0.0895512 ,  0.01832404,  0.08199107, -0.07083123,  0.03768105]])
      jac: array([-1.54972076e-06, -3.57627869e-07, -8.34465027e-07, -5.12599945e-06,
       -4.17232513e-06])
  message: 'Optimization terminated successfully.'
     nfev: 1033
      nit: 60
     njev: 147
   status: 0
  success: True
        x: array([ 4.74424872e+04,  3.34621475e+04,  5.52201513e+04,  5.30698979e+00,
       -9.72720093e-01])

In [21]:
import pandas as pd
train_data = pd.read_csv('../data/train_data.csv')
train_data.head()
X = train_data[['x','x_dot','F']]
D_x = X.values.shape[1]
theta_0 = np.random.normal(loc=2,scale = 1,size=D_x+2)
t = train_data[['x_prime','x_dot_prime']]
minimize(fun=neg_ll,x0=theta_0,args=(X.values,t.values,kernel_fn),method='L-BFGS-B')

      fun: -530.1244889056069
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.02240768, -0.18090986, -0.31222953,  0.01585931,  6.23232381])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 540
      nit: 38
   status: 0
  success: True
        x: array([ 1.27182150e+02,  8.74043931e+01,  8.49225624e+01,  3.49725494e+00,
       -1.40475178e-02])

we get the error <br>
"""
fun: -451.7428515599334
 hess_inv: array([[ 1.31958831e+01,  2.25244206e+01,  6.35074363e+01,
        -3.21234371e-01, -2.18691817e-03],
       [ 2.25244206e+01,  3.85101858e+01,  1.08902122e+02,
        -5.40652637e-01, -3.74693823e-03],
       [ 6.35074363e+01,  1.08902122e+02,  3.10097706e+02,
        -1.51666971e+00, -1.06503453e-02],
       [-3.21234371e-01, -5.40652637e-01, -1.51666971e+00,
         1.09053674e-02,  5.24254177e-05],
       [-2.18691817e-03, -3.74693823e-03, -1.06503453e-02,
         5.24254177e-05,  3.73928680e-07]])
      jac: array([ 2.96734238e+00, -6.96512985e+00, -5.29705048e-01,  4.49545670e+00,
       -4.81347301e+04])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 877
      nit: 20
     njev: 124
   status: 2
  success: False
        x: array([ 4.25329147e+01,  5.54329651e+01,  1.48233823e+02, -2.11552736e+00,
       -2.27977169e-04])
"""

In [15]:
def kernel_2(X1, X2, l=1.0, sigma_f=1.0):
    ''' Isotropic squared exponential kernel. Computes a covariance matrix from points in X1 and X2. Args: X1: Array of m points (m x d). X2: Array of n points (n x d). Returns: Covariance matrix (m x n). '''
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def nll_fn_2(X_train, Y_train, noise, naive=True):
    ''' Returns a function that computes the negative marginal log- likelihood for training data X_train and Y_train and given noise level. Args: X_train: training locations (m x d). Y_train: training targets (m x 1). noise: known noise level of Y_train. naive: if True use a naive implementation of Eq. (7), if False use a numerically more stable implementation. Returns: Minimization objective. '''
    def nll_naive(theta):
        # Naive implementation of Eq. (7). Works well for the examples 
        # in this article but is numerically less stable compared to 
        # the implementation in nll_stable below.
        K = kernel_2(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
            noise**2 * np.eye(len(X_train))
        nll_tot = 0
        for j in np.arange(Y_train.shape[1]):
            nll_j=0.5 * np.log(det(K)) + \
                        0.5 * Y_train[:,j].T.dot(inv(K).dot(Y_train[:,j])) + \
                        0.5 * len(X_train) * np.log(2*np.pi)
            nll_tot = nll_tot+nll_j
            return nll_tot

    def nll_stable_2(theta):
        # Numerically more stable implementation of Eq. (7) as described
        # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
        # 2.2, Algorithm 2.1.
        K = kernel_2(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
            noise**2 * np.eye(len(X_train))
        L = cholesky(K)
        return np.sum(np.log(np.diagonal(L))) + \
               0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
               0.5 * len(X_train) * np.log(2*np.pi)
    
    if naive:
        return nll_naive
    else:
        return nll_stable

In [20]:
noise = 0.4
#k = kernel_2(X1=X.values,X2=X.values)
nll_fn_2_ = nll_fn_2(X_train=X.values,Y_train=t.values,noise=noise)
nll_fn_2_([1,1])
res = minimize(nll_fn_2_, [1, 1], 
               bounds=((1e-5, None), (1e-5, None)),
               method='L-BFGS-B')
res

      fun: 16.363283882242314
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00014388, -0.00010907])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 105
      nit: 23
   status: 0
  success: True
        x: array([36.53374974, 16.2735757 ])

In [ ]:
# How to verify , plot predictive distribution for values from kernel 1 and kernel 2 , should be the same !
# is it called similarity verification !

## Ref:
1. http://krasserm.github.io/2018/03/19/gaussian-processes/
2. https://papers.nips.cc/paper/2420-gaussian-processes-in-reinforcement-learning.pdf